### Basic Hierarchical Chunking - ParentDocumentRetriever 📚🔍

warning 무시하기

In [ ]:
import warnings
warnings.filterwarnings("ignore")

#### Parent Document용 청킹과 Child Document용 청킹 선언

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

pdf_path = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

# 부모 문서용 텍스트 분할기(큰 청크)
parent_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len
)

# 자식 문서용 텍스트 분할기(작은 청크)
child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=50,
    length_function=len
)

#### Parent-Child Vector Store 선언하기

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain.retrievers import ParentDocumentRetriever
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.storage import InMemoryStore

embeddings = OllamaEmbeddings(model="bge-m3")

client = QdrantClient(":memory:")
# 벡터 저장소 초기화
client.create_collection(
    collection_name="hierarchical_chunks",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="hierarchical_chunks",
    embedding=embeddings,
)

# 부모 문서 저장소 초기화
store = InMemoryStore()

# ParentDocumentRetriever 설정
retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=store,
    child_splitter=child_splitter,
    parent_splitter=parent_splitter,
)

retriever.add_documents(documents)

#### Child Vector Store에서 문서 검색하기

In [ ]:
sub_docs = vector_store.similarity_search("영국 AI 정책")

print([len(sub_doc.page_content) for sub_doc in sub_docs])
sub_docs

#### ParentDocumentRetriever로 문서 검색하기

In [ ]:
retrieved_docs = retriever.invoke("영국 AI 정책")

print([len(doc.page_content) for doc in retrieved_docs])
retrieved_docs

### 📄 Markdown Hierarchical Chunking

In [ ]:
%pip install docling

#### 🔄 Docling으로 PDF --> 마크다운 변환하기

In [ ]:
from docling.document_converter import DocumentConverter

source = "data/국가별 공공부문 AI 도입 및 활용 전략.pdf"  # document per local path or URL
converter = DocumentConverter()
result = converter.convert(source)
result_document = result.document.export_to_markdown()

#### Markdown Header를 기준으로 Parent Chunking하기

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 헤더 정의 (헤더 레벨과 메타데이터 키 이름)
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
    ("####", "Header 4"),
]

# Markdown 헤더 기준 분할기 정의
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)

# 1단계: 헤더 기준으로 문서 분할
header_splits = markdown_splitter.split_text(result_document)
print([len(split.page_content) for split in header_splits])

#### ParentDocumentRetriever 구성하기

In [ ]:
from langchain_ollama import OllamaEmbeddings
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from langchain.storage import InMemoryStore
from langchain.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

embeddings = OllamaEmbeddings(model="bge-m3")

client = QdrantClient(":memory:")
# 벡터 저장소 초기화
client.create_collection(
    collection_name="hierarchical_chunks_markdown",
    vectors_config=VectorParams(size=1024, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="hierarchical_chunks_markdown",
    embedding=embeddings,
)

# 2단계: 각 헤더 섹션을 더 작은 청크로 분할
md_child_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=30,
    length_function=len
)

# 부모 문서 저장소 초기화 (Markdown용)
md_store = InMemoryStore()

# ParentDocumentRetriever 설정 (Markdown용)
md_retriever = ParentDocumentRetriever(
    vectorstore=vector_store,
    docstore=md_store,
    child_splitter=md_child_splitter,
    parent_splitter=None,  # 이미 헤더로 분할했으므로 부모 분할기는 필요 없음
    search_kwargs={"k": 5}
)

# 문서 추가 - 헤더로 분할된 문서를 부모 문서로 사용
md_retriever.add_documents(header_splits)

#### Child Chunk를 VectorDB에서 검색해보기

In [ ]:
sub_docs=vector_store.similarity_search("영국 중앙 디지털데이터청(CDDO)의 역할과 그 전략적 파트너 기관은 누구이며, 함께 수립한 전략의 주요 목표는 무엇인가?")
print([len(doc.page_content) for doc in sub_docs])
print('\n'.join([doc.page_content for doc in sub_docs]))

#### ParentDocumentRetriever로 문서 검색해보기

In [ ]:
retrieved_docs=md_retriever.invoke("대한민국 공공부문에서 초거대 AI의 실질적 도입을 가능하게 하는 인프라, 거버넌스, 실행계획은 어떻게 유기적으로 연계되어 있는가?")
print([len(doc.page_content) for doc in retrieved_docs])
print('\n'.join([doc.page_content for doc in retrieved_docs]))

In [ ]:
[doc.metadata for doc in retrieved_docs]